In [ ]:
# default_exp core

# Core

> Conatains methods for transferring.

In [ ]:
#hide
from nbdev.showdoc import *
from fastai.tabular.all import *

In [ ]:
# This will contain the functions for transferring embeddings

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df1 = pd.read_csv(path/'adult.csv')
splits1 = RandomSplitter(valid_pct=0.2)(range_of(df1))
to1 = TabularPandas(df1, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits1)
dls1 = to1.dataloaders(bs=64)
learn1 = tabular_learner(dls1, metrics=accuracy)

In [ ]:
learn1.model.embeds

ModuleList(
  (0): Embedding(10, 6)
  (1): Embedding(17, 8)
  (2): Embedding(8, 5)
  (3): Embedding(16, 8)
  (4): Embedding(7, 5)
  (5): Embedding(6, 4)
  (6): Embedding(3, 3)
)

Aim is to take these embeddings and transfer them to other models

In [ ]:
learn1.model.embeds[0].weight, learn1.model.embeds[0].weight.shape

(Parameter containing:
 tensor([[ 8.6823e-03, -5.9003e-04, -9.9109e-03, -5.9177e-03,  1.5016e-02,
           5.3239e-03],
         [ 6.9098e-03, -9.9316e-03,  3.0800e-03,  2.6611e-03,  3.5328e-03,
           1.4779e-02],
         [-1.3377e-02, -4.5740e-03, -7.3343e-03,  1.7469e-04,  1.6012e-02,
           5.0647e-03],
         [ 1.3192e-02,  1.8187e-02,  2.7549e-03,  1.0228e-03,  2.0640e-03,
           4.4759e-05],
         [-1.5011e-02,  1.8566e-02,  9.2129e-04, -1.0403e-02, -1.0669e-02,
          -1.5862e-03],
         [-3.1854e-03,  6.8827e-03,  5.7019e-03,  7.7152e-06,  1.7651e-02,
          -8.7786e-04],
         [-2.6262e-03, -2.3546e-03, -1.4675e-02,  1.4537e-02,  8.8957e-03,
           3.7119e-03],
         [ 1.6608e-03, -6.1384e-03, -1.8192e-03,  5.2454e-03, -3.9614e-03,
          -1.4462e-02],
         [ 1.7626e-03,  1.0131e-02,  1.9187e-03,  3.7003e-03,  3.8059e-03,
          -2.2213e-03],
         [-1.6983e-04, -1.2205e-02, -1.7088e-02,  1.5608e-02, -7.3543e-03,
          -

In [ ]:
#We add robot to our "race" column
new_rows = pd.DataFrame([[49,'Private',101320,'Assoc-acdm',12.0,'Married-civ-spouse','Exec-managerial','Wife','Robot','Female',0,1902,40,'United-States','>=50k'],
                        [18,'Private',182308,'Bachelors',10.0,'Never-married','?','Own-child','Other','Male',0,0,23,'United-States','<50k']],
                        columns=df1.columns)
df2 = df1.copy()
df2 = df2.append(new_rows, ignore_index=True)
df2.tail()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
32558,53,Private,157069,Assoc-acdm,12.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>=50k
32559,32,Local-gov,217296,HS-grad,9.0,Married-civ-spouse,Transport-moving,Wife,White,Female,4064,0,22,United-States,<50k
32560,26,Private,182308,Some-college,10.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,<50k
32561,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,Exec-managerial,Wife,Robot,Female,0,1902,40,United-States,>=50k
32562,18,Private,182308,Bachelors,10.0,Never-married,?,Own-child,Other,Male,0,0,23,United-States,<50k


In [ ]:
splits2 = RandomSplitter(valid_pct=0.2)(range_of(df2))
to2 = TabularPandas(df2, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits2)
dls2 = to2.dataloaders(bs=64)
learn2 = tabular_learner(dls2, metrics=accuracy)

In [ ]:
learn2.model.embeds

ModuleList(
  (0): Embedding(11, 6)
  (1): Embedding(19, 8)
  (2): Embedding(10, 6)
  (3): Embedding(18, 8)
  (4): Embedding(9, 5)
  (5): Embedding(8, 5)
  (6): Embedding(3, 3)
)

In [ ]:
learn1.dls.cat_names

(#7) ['workclass','education','marital-status','occupation','relationship','race','education-num_na']

In [ ]:
learn1.dls.classes

{'workclass': ['#na#', ' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'],
 'education': ['#na#', ' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college'],
 'marital-status': ['#na#', ' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed'],
 'occupation': ['#na#', ' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving'],
 'relationship': ['#na#', ' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife'],
 'race': ['#na#', ' Amer-Indian-Eskimo', ' Asian-Pac-Island

In [ ]:
learn1.fit(1)

epoch     train_loss  valid_loss  accuracy  time    
0         0.373516    0.354076    0.833999  00:04     


In [ ]:
temp = 1

In [ ]:
#export
class TabTransfer:
    def __init__(self, old_learner, new_learner, cat_names_to_transfer):
        self.old_cat_names = old_learner.dls.cat_names
        self.old_all_classes = old_learner.dls.classes
        self.old_learner = old_learner
        
        self.new_cat_names = new_learner.dls.cat_names
        self.new_all_classes = new_learner.dls.classes
        self.new_learner = new_learner
        
        self.transfer_list = cat_names_to_transfer
        
        
    def transfer(self):
        for curr_cat in self.transfer_list:
            if not (curr_cat in self.old_cat_names and curr_cat in self.new_cat_names):
                continue
            old_cat_idx = self.old_cat_names.index(curr_cat)
            new_cat_idx = self.new_cat_names.index(curr_cat)
            
            
            
            # TODO: Make it so that this isn't required by taking care of this.
            try: assert (self.old_learner.model.embeds[old_cat_idx].embedding_dim == self.new_learner.model.embeds[new_cat_idx].embedding_dim)
            except: 
                print(f"Encountered an error for variable {curr_cat}: Make sure embeddings dimensions are same for {self.old_learner.model.embeds[old_cat_idx]} and {self.new_learner.model.embeds[new_cat_idx]}")
                print("Moving on to other cat vars")
                continue
            
            old_curr_classes = self.old_all_classes[curr_cat]
            new_curr_classes = self.new_all_classes[curr_cat]
            global temp
            temp = new_curr_classes
#             if old_curr_classes<new_curr_classes:
#                 sitch = 1
#             else if old_curr_classes=new_curr_classes:
#                 sitch = 2
#             else if old_curr_classes<new_curr_classes:
#                 sitch = 3
            
            weights_mean = self.old_learner.model.embeds[old_cat_idx].weight.mean(0)
            print(f'mean is {weights_mean} for {self.old_learner.model.embeds[old_cat_idx].weight}')
                
#             switch(sitch):
#                 case 1: 

            # Case where some category in old, but not in new isn't being handled rn.

            for new_curr_class in new_curr_classes:
                new_curr_class_idx = new_curr_classes.o2i[new_curr_class]
                print(f"{new_curr_class_idx}, {type(new_curr_class_idx)}")
                
                if new_curr_class in old_curr_classes:
                    old_curr_class_idx = old_curr_classes.o2i[new_curr_class]
                    print(f'Transferring weights for class {new_curr_class}, cat {curr_cat} from previous weights')
                    print(f"old weight for class is {self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]}")
                    tempwgt1 = self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]
                    tempwgt2 = self.old_learner.model.embeds[old_cat_idx].weight[old_curr_class_idx, :]
                    self.new_learner.model.embeds[new_cat_idx].weight.data[new_curr_class_idx, :] = self.old_learner.model.embeds[old_cat_idx].weight[old_curr_class_idx, :].detach().clone()
                    self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :].required_grad = True
                    print(f"new weight for class is {self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]}")
                else:
                    print(f'Transferring weights for class {new_curr_class}, cat {curr_cat} using mean')
                    print(f"old weight for class is {self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]}")
                    self.new_learner.model.embeds[new_cat_idx].weight.data[new_curr_class_idx, :] = weights_mean
                    self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :].required_grad = True
                    print(f"new weight for class is {self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]}")

In [ ]:
learn1.model.embeds[0].embedding_dim

6

In [ ]:
tabobj = TabTransfer(learn1, learn2, ["race", "workclass"])

In [ ]:
tabobj.transfer()

Encountered an error for variable race: Make sure embeddings dimensions are same for Embedding(6, 4) and Embedding(8, 5)
Moving on to other cat vars
mean is tensor([-0.0003,  0.0005, -0.0002,  0.0090,  0.0007, -0.0046],
       grad_fn=<MeanBackward1>) for Parameter containing:
tensor([[ 0.0086, -0.0006, -0.0099, -0.0059,  0.0150,  0.0053],
        [ 0.0305, -0.0044, -0.0034,  0.0234,  0.0180,  0.0237],
        [-0.0570, -0.0361,  0.0253, -0.0568,  0.0298, -0.0370],
        [ 0.0185,  0.0249, -0.0102,  0.0268, -0.0218, -0.0109],
        [-0.0308,  0.0017,  0.0057, -0.0073, -0.0129, -0.0024],
        [-0.0190, -0.0027,  0.0249, -0.0041,  0.0343,  0.0041],
        [-0.0700, -0.0561,  0.0320, -0.0512,  0.0422, -0.0471],
        [ 0.0491,  0.0339, -0.0586,  0.0735, -0.0454,  0.0039],
        [ 0.0387,  0.0261,  0.0005,  0.0382, -0.0259,  0.0210],
        [ 0.0281,  0.0185, -0.0081,  0.0533, -0.0263, -0.0061]],
       requires_grad=True)
0, <class 'int'>
Transferring weights for class #na#, 